In [62]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [63]:
# Complete project details at https://RandomNerdTutorials.com

try:
  import usocket as socket
except:
  import socket

from machine import Pin
import network

import esp
esp.osdebug(None)

import utime

import gc
gc.collect()



In [64]:
sta_if = network.WLAN(network.STA_IF)
ap_if = network.WLAN(network.AP_IF)
print('Station Up: ', sta_if.active())
print('Acess Point Up: ', ap_if.active())

Station Up:  False
Acess Point Up:  False


In [65]:
def connectX(typeX):
    typeX.active(True)
    print('Station Up: ', sta_if.active())
    print('Acess Point Up: ', ap_if.active())
    
def scanX():
    print('Status: %s\n' % sta_if.status())
    station_default = False
    stations_list = sta_if.scan()
    for i, n in enumerate(stations_list, start=0):
        print('%s) %s' %(i, n))
        if (n[0]) == b'SM-A1kerr':
            station_default = True

    if station_default == False:
        print('*** PLEASE ENABLE DEFAULT STATION! ***')
    return stations_list
    

In [66]:
connectX(sta_if)
stations_list = []


if sta_if.active():
#     print('Station Up: ', sta_if.active())
    stations_list = scanX()
#     print(stations_list)
#     print(sta_if.status())
elif ap_if.active():
    print('Acess Point Up: ', ap_if.active())

Station Up:  True
Acess Point Up:  False
Status: 1000

0) (b'SM-A1kerr', b'\x8e\xf5\xa3\xa2\x7fn', 1, -33, 3, False)
1) (b'DIRECT-AD-HP OfficeJet Pro 7740', b'\x1a`$i)\xad', 6, -65, 3, False)
2) (b'Sheridan Secure Access', b"\xa8\xbd'\x94d ", 11, -73, 5, False)
3) (b'Staff Wireless', b"\xa8\xbd'\x94d!", 11, -73, 5, False)
4) (b'Public Wireless', b'\xa8\xbd\'\x94d"', 11, -73, 0, False)
5) (b'Sheridan Secure Access', b"\xa8\xbd'\x94o@", 6, -77, 5, False)
6) (b'Sheridan Secure Access', b"\xa8\xbd'\x94c\xc0", 1, -86, 5, False)
7) (b'Sheridan Secure Access', b'\xb0\xb8g\xbdz\x00', 1, -86, 5, False)
8) (b'Public Wireless', b'\xb0\xb8g\xbdz\x02', 1, -86, 0, False)
9) (b'Staff Wireless', b"\xa8\xbd'\x93AA", 11, -87, 5, False)
10) (b'Sheridan Secure Access', b"\xa8\xbd'\x93A@", 11, -88, 5, False)


In [67]:
print('Connecting to: ', stations_list[0][0])
while not(sta_if.isconnected()):
    print('Trying to connet to server!')
    sta_if.connect(stations_list[0][0], '9054830760')
    utime.sleep(2)

print('Status: %s\n' % sta_if.status())
print('Is connected: ', sta_if.isconnected())
print('Connected to: ', sta_if.config('essid'))
print('ifconfig: ', sta_if.ifconfig())


Connecting to:  b'SM-A1kerr'
Trying to connet to server!
Trying to connet to server!
.Status: 1010

Is connected:  True
Connected to:  SM-A1kerr
ifconfig:  ('192.168.43.34', '255.255.255.0', '192.168.43.1', '192.168.43.1')


<br>
<p>
Format of getaddrinfo:  [(2, 1, 0, '192.168.43.34', ('192.168.43.34', 80))]
</p>

In [68]:
# Complete project details at https://RandomNerdTutorials.com

import socket
addr_socket = socket.getaddrinfo('192.168.43.34', 80)[0][4]
# addr = socket.getaddrinfo('192.168.43.34', 80)
led = Pin(2, Pin.OUT)

def web_page():
  if led.value() == 1:
    gpio_state="ON"
  else:
    gpio_state="OFF"
  
  html = """<html><head> <title>ESP Web Server</title> <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" href="data:,"> <style>html{font-family: Helvetica; display:inline-block; margin: 0px auto; text-align: center;}
  h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display: inline-block; background-color: #e7bd3b; border: none; 
  border-radius: 4px; color: white; padding: 16px 40px; text-decoration: none; font-size: 30px; margin: 2px; cursor: pointer;}
  .button2{background-color: #4286f4;}</style></head><body> <h1>ESP Web Server</h1> 
  <p>GPIO state: <strong>""" + gpio_state + """</strong></p><p><a href="/?led=on"><button class="button">ON</button></a></p>
  <p><a href="/?led=off"><button class="button button2">OFF</button></a></p></body></html>"""
  return html

html2 = """<!DOCTYPE html>
<html>
    <head> <title>ESP8266 Pins</title> </head>
    <body> <h1>ESP8266 Pins</h1>
        <table border="1"> <tr><th>Pin</th><th>Value</th></tr> %s </table>
    </body>
</html>
"""

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# s.bind(('', 80))
# s.bind(addr)
s.bind(('192.168.43.34', 80))
print('Bind: ', addr_socket)
# print('Bind: %s' % addr[0])
# print('Bind: %s' % addr[0][-1])
s.listen(5)

while True:
    # #      clientX = s.accept()
    conn, addr = s.accept()
    print('Got a connection!: ')
    #      print('Address: %s' % addr[0])
    #      print('Address: %s' % addr[1])
    # #      print(clientX[1])
    # #      print(' %s' % str(addr))
    print('Got a connection from %s' % str(addr))
    request = conn.recv(1024)
    request = str(request)
    print('Content = %s' % request)
    conn.send("I am SERVER\n")
    conn.close()
    #   led_on = request.find('/?led=on')
    #   led_off = request.find('/?led=off')
    #   if led_on == 6:
    #     print('LED ON')
    #     led.value(1)
    #   if led_off == 6:
    #     print('LED OFF')
    #     led.value(0)
    #   response = web_page()
#      conn.send('HTTP/1.1 200 OK\n')
#      conn.send('Content-Type: text/html\n')
#      conn.send('Connection: close\n\n')
    #   conn.sendall(response)
print('Server closed: ')
        

Bind:  ('192.168.43.34', 80)
.Got a connection!: 
Got a connection from ('192.168.43.84', 50512)
Content = b'Twenty-five bytes to send'
..........................Got a connection!: 
Got a connection from ('192.168.43.84', 50866)
Content = b'\x16\x03\x01\x02\x00\x01\x00\x01\xfc\x03\x03\x99_\xc0\x16\x17\xc1\x0e\xb6\xa4m\x8b&\xc5\xb4>\x16\x86}\xea\xe1\x7fO\x065\x08_\x1a\x85+\xfeW\xa9 7d\xe1\x0e\xee\x14\xb9\x85L\\hqE\x86\x05\xb5\xad\x93q8e\x8d\xf6\xe7\xe2[\xd7\xcdp\xc3+\xc3\x00"\n\n\x13\x03\x13\x01\x13\x02\xcc\xa9\xcc\xa8\xc0+\xc0/\xc0,\xc00\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x00\n\x01\x00\x01\x91JJ\x00\x00\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\n\x00\x08\x9a\x9a\x00\x1d\x00\x17\x00\x18\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\r\x00\x14\x00\x12\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x02\x01\x00\x12\x00\x00\x003\x00+\x00)\x9a\x9a\x00\x01\x00\x00\x1d\x00 `E{\xf7\x98\x11\xb4p\xc7\xd4\xe4\xf0%\xc9\xfb\x9a\xa1\x98\x89\x0

Traceback (most recent call last):
  File "<stdin>", line 43, in <module>
KeyboardInterrupt: 
